# HANC with a Welfare State

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Test 1: Solving and simulating the household problem](#toc2_)    
- 3. [Test 2: Evaluating the objective for finding the steady state](#toc3_)    
- 4. [Find stationary equilibrium](#toc4_)    
- 5. [Grid search](#toc5_)    
- 6. [Policy functions](#toc6_)    
- 7. [Simulation](#toc7_)    
- 8. [Test transition path](#toc8_)    
- 9. [Find transition path](#toc9_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize
from scipy.optimize import minimize_scalar

import matplotlib.pyplot as plt   
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

from HANCWelfareModel import HANCWelfareModelClass
from steady_state import obj_ss

from mpl_toolkits.mplot3d import Axes3D


## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [ ]:
model = HANCWelfareModelClass(name='baseline')

par = model.par
ss = model.ss
sol = model.sol
path = model.path

## 2. <a id='toc2_'></a>[Test 1: Solving and simulating the household problem](#toc0_)

We set all the steady values for the household problem manually at ad hoc values:

In [ ]:
ss.r = 0.02*(1-0.1)
ss.wt = 1.00*(1-0.3)
ss.S = 0.0
ss.Chi = 0.1
ss.G = 0.4

model.solve_hh_ss(do_print=True)
model.simulate_hh_ss(do_print=True)


In [ ]:
par.Chi_ = 0.1
par.G_ = 0.4
model.find_ss(do_print=True)


In [ ]:
def print_ss():
    print(f'Steady state values:')
    for varname in model.varlist:
        print(f'{varname:15s}: {ss.__dict__[varname]:.4f}')
print_ss()

In [ ]:
model.test_path(in_place=True) 

In [ ]:
print(path.__dict__['clearing_Y'][:10])

In [ ]:
model.compute_jacs(do_print=True)

In [ ]:
model.find_transition_path(shocks=[],do_print=True)

# Question 1 Stationary Equalibrium without gov 

### 1.1 Steady state values

In [ ]:
par.G_ = 0.0
par.Chi_ = 0.0
model.find_ss(do_print=False)
print_ss()

In [ ]:
def udskriv():
    with open('output_table.tex', 'w') as file:
        file.write("\\begin{tabular}{l r}\n")
        file.write("\\hline\n")
        file.write("Variable Name & Value \\\\\n")
        file.write("\\hline\n")
        for varname in model.varlist:
            file.write(f'{varname} & {ss.__dict__[varname]:.4f} \\\\\n')
        file.write("\\hline\n")
        file.write("\\end{tabular}\n")
#udskriv()

### 1.2 Policy functions

In [ ]:
i_fix = 0

fig = plt.figure(figsize=(18,4),dpi=100)
a_max = 100

# a. consumption
I = par.a_grid < a_max

ax = fig.add_subplot(1,3,1)
ax.set_title(f'consumption')

for i_z in [0,par.Nz//2,par.Nz-1]:
    ax.plot(par.a_grid[I],ss.c[i_fix,i_z,I],label=f'i_z = {i_z}')

ax.legend(frameon=True)
ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('consumption, $c_t$')

# b. saving
I = par.a_grid < a_max

ax = fig.add_subplot(1,3,2)
ax.set_title(f'saving')

for i_z in [0,par.Nz//2,par.Nz-1]:
    ax.plot(par.a_grid[I],ss.a[i_fix,i_z,I],label=f'i_z = {i_z}')

ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('savings, $a_{t}$')

# c. labor supply
I = par.a_grid < a_max

ax = fig.add_subplot(1,3,3)
ax.set_title(f'labor_supply')

for i_z in [0,par.Nz//2,par.Nz-1]:
    ax.plot(par.a_grid[I],ss.ell[i_fix,i_z,I],label=f'i_z = {i_z}')

ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('labor supply, $n_{t}$')


fig.tight_layout()
#save figure
#fig.savefig('figs/fig_policy.png', bbox_inches='tight')

plt.show()

### 1.3 Distribution in ss 

In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)

# a. income
ax = fig.add_subplot(1,2,1)
ax.set_title('productivity')

y = np.cumsum(np.sum(ss.D,axis=(0,2)))
ax.plot(par.z_grid,y/y[-1])

ax.set_xlabel('productivity, $z_{t}$')
ax.set_ylabel('CDF')

# b. assets
ax = fig.add_subplot(1,2,2)
ax.set_title('savings')
y = np.insert(np.cumsum(np.sum(ss.D,axis=(0,1))),0,0.0)
ax.plot(np.insert(par.a_grid,0,par.a_grid[0]),y/y[-1])
        
ax.set_xlabel('assets, $a_{t}$')
ax.set_ylabel('CDF')
ax.set_xscale('symlog')

#save figure
#fig.savefig('figs/fig_distribution.png', bbox_inches='tight')
plt.show()

### 1.4 Expected discounted utility

In [ ]:
#Ploting the accumulation discounted utility
#par.T = 500
time_ = [1, 10, 20, 30, 40, 50,  100, 150, 200, 250, 300, 350, 400, 450, 500]

#time_ = [1, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500]


time_past = []
disc_utility = []
for i in time_:
    par.T = i 
    time_past.append(i)
    U =np.sum([par.beta**t * np.sum(path.u[t]*path.D[t]/np.sum(path.D[t])) for t in range(par.T)])
    disc_utility.append(U)


print(f'Utility: {disc_utility[-0]:.4f}')

#figure 
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1,1,1)
ax.plot(time_past, disc_utility, label='Discounted Utility')
ax.set_xlabel('Time')
ax.set_ylabel('Utility')
ax.legend()

# save figure
#fig.savefig('figs/fig_disc_utility.png', bbox_inches='tight')
#print(disc_utility)
plt.show()



# 2. Optimal welfare policies 1 (no transfers)

### 2.1 Defining new functions

In [ ]:
# New objetctiv function 
def u_gov(x):
    """Utility function given government"""
    par.G_ = x
    model.find_ss(do_print=False)

    #model.find_transition_path(shocks=[],do_print=False)
    model.test_path(in_place=False) 
    model.compute_jacs(do_print=False)
    model.find_transition_path(shocks=[],do_print=False)

    U =np.sum([par.beta**t * np.sum(path.u[t]*path.D[t]/np.sum(path.D[t])) for t in range(par.T)])
    return U

def obj_gov(x):
    """Objective function with government as the choice variable"""
    return - u_gov(x)

u_gov(0.1)

In [ ]:
# Optimize the objective function within the specified range
par.Chi_ = 0.0

result = minimize_scalar( obj_gov, bounds=(0.375, 0.45), method='bounded')

# Print the optimization result
print(f"Optimal value of x: {- result.x}, given Chi = {ss.Chi}")
print(f"Optimal objective function value: {- result.fun}")
print(f"G/Y in the steady state: {ss.G/ss.Y}")


In [ ]:

#result = minimize_scalar( obj_gov, bounds=(0.375, 0.45), method='bounded')

g_values = np.linspace(0.3, 0.5, 10)
u_values = [u_gov(x) for x in g_values]

fig = plt.figure(figsize=(7, 7), dpi=100)
ax = fig.add_subplot(1, 1, 1)
ax.set_title('Title')

# Plot the gov_u function
ax.plot(g_values, u_values, label='Objective Function')
ax.set_xlabel('Government production')
ax.set_ylabel('Discounted expected utility')
ax.set_title('Plot of Objective Function')
ax.grid(True)
ax.legend()

# Optimal government consumption
ax.scatter(result.x, u_gov(result.x), color='red', label='Optimal government consumption', marker='o')
ax.legend()

fig.savefig('figs/fig_opt_gov.png', bbox_inches='tight')

### 2.2 Solving for the optimal government production

In [ ]:
u_val = [i ]

In [156]:
#chi_val = []
#gov_val = []
#u_val = []
#y_G_relatio = []

par.Chi_ = 0.1
result = minimize_scalar( obj_gov, bounds=(0.375, 0.45), method='bounded')
print(f"Solution found")
chi_val.append(par.Chi_ )
gov_val.append(ss.G)
y_G_relatio.append(ss.G/ss.Y)
print(f'Første done')

shocks: G L_G Gamma_G 
unknowns: K L_Y 

blocks.production_firm
 rK               0.0e+00
 w                0.0e+00
 Y                0.0e+00
blocks.mutual_fund
 A                0.0e+00
 r                0.0e+00
blocks.government
 tau              0.0e+00
 wt               0.0e+00
 Chi              0.0e+00
 S                0.0e+00
hh
 A_hh             1.2e-09
 C_hh             3.5e-11
 ELL_hh           1.6e-10
 L_hh             1.3e-10
 INC_hh           9.2e-12
 U_hh             3.7e-10
blocks.market_clearing
 L                1.3e-10
 I                2.8e-16
 clearing_A       1.2e-09 [target]
 clearing_L       1.3e-10
 clearing_Y       3.5e-11
 clearing_G       1.1e-10 [target]


/Users/emmagundersen/Library/Mobile Documents/com~apple~CloudDocs/KU/9 semester/Het_assignments/Assignment_2/Assignment_II/steady_state.py:90: RuntimeWarning: invalid value encountered in double_scalars
  ss.Y = par.Gamma_Y*ss.K**(par.alpha)*ss.L_Y**(1-par.alpha) # Total privat production


shocks: G L_G Gamma_G 
unknowns: K L_Y 

blocks.production_firm
 rK               0.0e+00
 w                0.0e+00
 Y                0.0e+00
blocks.mutual_fund
 A                0.0e+00
 r                0.0e+00
blocks.government
 tau              0.0e+00
 wt               0.0e+00
 Chi              0.0e+00
 S                0.0e+00
hh
 A_hh             1.3e-09
 C_hh             3.8e-11
 ELL_hh           1.8e-10
 L_hh             1.5e-10
 INC_hh           1.2e-11
 U_hh             4.5e-10
blocks.market_clearing
 L                1.5e-10
 I                8.3e-17
 clearing_A       1.3e-09 [target]
 clearing_L       1.5e-10
 clearing_Y       3.8e-11
 clearing_G       1.2e-10 [target]
shocks: G L_G Gamma_G 
unknowns: K L_Y 

blocks.production_firm
 rK               0.0e+00
 w                0.0e+00
 Y                0.0e+00
blocks.mutual_fund
 A                0.0e+00
 r                0.0e+00
blocks.government
 tau              0.0e+00
 wt               0.0e+00
 Chi              0.0e+00


In [158]:
with open('output_gov_chi.tex', 'w') as file:
    file.write("\\begin{tabular}{c c c c}\n")
    file.write("\\hline\n")
    file.write("Chi & Government & U & Y/G \\\\\n")
    file.write("\\hline\n")
    for i in range(len(chi_val)):
        file.write(f"{chi_val[i]} & {gov_val[i]:.4f} & {u_val[i]:.4f} & {y_G_relatio[i]:.4f} \\\\\n")
    file.write("\\hline\n")
    file.write("\\end{tabular}\n")

In [159]:
print(f'Chi: {chi_val}')
print(f'Gov: {gov_val}')
print(f'U: {u_val}')
print(f'Y/G: {y_G_relatio}')


Chi: [-0.1, -0.1, 0.1]
Gov: [0.44999531775951596, 0.44999531775951596, 0.44999531775951596]
U: [-143.80657876435623, -143.80657876435623, -143.80657876435623]
Y/G: [0.3421417394590169, 0.3687678190319306, 0.4295822803060873]


In [151]:

chi_val = [-0.1, 0.0, 0.1]
gov_val = []
u_val = []
y_G_relatio = []
for i in chi_val:
    par.Chi_ = i
    result = minimize_scalar( obj_gov, bounds=(0.375, 0.45), method='bounded')
    chi_val.append(i)
    gov_val.append(result.x)
    u_val.append(-result.fun)
    y_G_relatio.append(ss.G/ss.Y)



shocks: G L_G Gamma_G 
unknowns: K L_Y 

blocks.production_firm
 rK               0.0e+00
 w                0.0e+00
 Y                0.0e+00
blocks.mutual_fund
 A                0.0e+00
 r                0.0e+00
blocks.government
 tau              0.0e+00
 wt               0.0e+00
 Chi              0.0e+00
 S                0.0e+00
hh
 A_hh             9.0e-10
 C_hh             2.3e-11
 ELL_hh           1.0e-10
 L_hh             8.4e-11
 INC_hh           4.2e-12
 U_hh             2.3e-10
blocks.market_clearing
 L                8.4e-11
 I                5.6e-17
 clearing_A       7.6e-10 [target]
 clearing_L       8.4e-11
 clearing_Y       2.3e-11
 clearing_G       1.5e-10 [target]
shocks: G L_G Gamma_G 
unknowns: K L_Y 

blocks.production_firm
 rK               0.0e+00
 w                0.0e+00
 Y                0.0e+00
blocks.mutual_fund
 A                0.0e+00
 r                0.0e+00
blocks.government
 tau              0.0e+00
 wt               0.0e+00
 Chi              0.0e+00


KeyboardInterrupt: 

In [ ]:
with open('output_gov_chi.tex', 'w') as file:
    file.write("\\begin{tabular}{c c c c}\n")
    file.write("\\hline\n")
    file.write("Chi & Government & U & Y/G \\\\\n")
    file.write("\\hline\n")
    for i in range(len(chi_val)):
        file.write(f"{chi_val[i]} & {gov_val[i]:.4f} & {u_val[i]:.4f} & {y_G_ratio[i]:.4f} \\\\\n")
    file.write("\\hline\n")
    file.write("\\end{tabular}\n")

In [ ]:
# Optimize the objective function within the specified range
par.Chi_ = 0.1

result = minimize_scalar( obj_gov, bounds=(0.375, 0.45), method='bounded')

# Print the optimization result
print(f"Optimal value of x: {- result.x}, given Chi = {ss.Chi}")
print(f"Optimal objective function value: {- result.fun}")
print(f"G/Y in the steady state: {ss.G/ss.Y}")


### 2.3 Ploting the utility for given values of g

In [ ]:

par.Chi_ = 0.1

#result = minimize_scalar( obj_gov, bounds=(0.375, 0.45), method='bounded')

g_values = np.linspace(0.3, 0.5, 10)
u_values = [u_gov(x) for x in g_values]

fig = plt.figure(figsize=(7, 7), dpi=100)
ax = fig.add_subplot(1, 1, 1)
ax.set_title('Title')

# Plot the gov_u function
ax.plot(g_values, u_values, label='Objective Function')
ax.set_xlabel('Government production')
ax.set_ylabel('Discounted expected utility')
ax.set_title('Plot of Objective Function')
ax.grid(True)
ax.legend()

# Optimal government consumption
ax.scatter(result.x, u_gov(result.x), color='red', label='Optimal government consumption', marker='o')
ax.legend()

# 3. Find optimal welfare policies II (with transfers)

In [ ]:

# Your obj_gov and u_gov functions should be defined here

# Define g_values
g_values = np.linspace(0.3, 0.5, 20)

# Initialize plot
fig = plt.figure(figsize=(7, 7), dpi=100)
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Government production')
ax.set_ylabel('Discounted expected utility')
ax.set_title('Plot of Objective Function')
ax.grid(True)

# First scenario: Chi_ = 0.0
par.Chi_ = 0.0
result = minimize_scalar(obj_gov, bounds=(0.35, 0.45), method='bounded')
u_values = [u_gov(x) for x in g_values]
ax.plot(g_values, u_values, label='Transfers = 0.0')
ax.scatter(result.x, u_gov(result.x), color='red', marker='o')

# Second scenario: Chi_ = 0.1
par.Chi_ = 0.1
result = minimize_scalar(obj_gov, bounds=(0.35, 0.45), method='bounded')
u_values = [u_gov(x) for x in g_values]
ax.plot(g_values, u_values, label='Transfers = 0.1')
ax.scatter(result.x, u_gov(result.x), color='blue', marker='o')


# Second scenario: Chi_ = 0.1
par.Chi_ = - 0.1
result = minimize_scalar(obj_gov, bounds=(0.35, 0.45), method='bounded')
u_values = [u_gov(x) for x in g_values]
ax.plot(g_values, u_values, label='Transfers = - 0.1')
ax.scatter(result.x, u_gov(result.x), color='blue', marker='o')

# Add legend
ax.legend()

# Show plot
ax.set_title('')

#fig.savefig('figs/disc_ut_gov_chi.png', bbox_inches='tight')
plt.show()


In [ ]:
par.Chi_ = 0.1

print(f'Chi: {ss.Chi}')
result = minimize_scalar( obj_gov, bounds=(0.3, 0.5), method='bounded')
# Print the optimization result
print(f"Optimal value of x: {- result.x}, given Chi = {ss.Chi}")
print(f"Optimal objective function value: {- result.fun}")
print(f"G/Y in the steady state: {ss.G/ss.Y}")


print(f'\n\n')

print(f'Chi: {ss.Chi}')
par.Chi_ = - 0.1
result = minimize_scalar( obj_gov, bounds=(0.3, 0.5), method='bounded')
# Print the optimization result
print(f"Optimal value of x: {- result.x}, given Chi = {ss.Chi}")
print(f"Optimal objective function value: {- result.fun}")
print(f"G/Y in the steady state: {ss.G/ss.Y}")



# STOP 

In [ ]:
# Error  RuntimeWarning: invalid value encountered in double_scalars
# Create the figure and subplots
fig = plt.figure(figsize=(10, 10), dpi=100)

#
ax = fig.add_subplot(1, 1, 1)
ax.set_title('Productivity')


g_values = np.linspace(0.3, 0.5, 20)
u_values = [u_gov(x) for x in g_values]


# Plot the objective function
ax.plot(g_values, u_values, label='Objective Function')
ax.set_xlabel('Government production')
ax.set_ylabel('Discounted expected utility')
ax.set_title('Plot of Objective Function')
ax.grid(True)
ax.legend()

# Plot the minimum point
ax.scatter(result.x, u_gov(result.x), color='red', label='Optimal government consumption', marker='o')
ax.legend()

# Save figure
fig.savefig('figs/fig_optimal_G.png', bbox_inches='tight')

# Show the figure
plt.show()


## Question 3

In [ ]:
# Optimize the objective function within the specified range
par.Chi_ss = 0.05

result = minimize_scalar( obj_gov, bounds=(0.375, 0.45), method='bounded')

# Print the optimization result
print(f"Optimal value of x: {- result.x}, given Chi = {ss.Chi}")
print(f"Optimal objective function value: {- result.fun}")

# G/Y in the steady state
# PROBLEM - G in the steady state: 1.22 skal være 0.41493
print(f"G/Y in the steady state: {ss.G/ss.Y}")
print(f"L_G in the steady state: {ss.L_G}")
print(f"G in the steady state: {ss.Y}")
print(f"Y in the steady state: {ss.Gamma_G}")
print(f'S in the steady state: {ss.S}')
print(f'Chi in the steady state: {ss.Chi}')
print(f'U_hh in the steady state: {ss.U_hh}')
print(f'G_ss in the steady state: {par.G_ss}')

In [ ]:
# Error  RuntimeWarning: invalid value encountered in double_scalars
# Create the figure and subplots
fig = plt.figure(figsize=(10, 10), dpi=100)

#
ax = fig.add_subplot(1, 1, 1)
ax.set_title('Productivity')

g_values = np.linspace(0.3, 0.5, 10)
u_values = [u_gov(x) for x in g_values]


# Plot the objective function
ax.plot(g_values, u_values, label='Objective Function')
ax.set_xlabel('Government production')
ax.set_ylabel('Discounted expected utility')
ax.set_title('Plot of Objective Function')
ax.grid(True)
ax.legend()

# Plot the minimum point
ax.scatter(result.x, u_gov(result.x), color='red', label='Optimal government consumption', marker='o')
ax.legend()

# Save figure
#fig.savefig(f'figs/fig_optimal_G_{ss.Chi}.png', bbox_inches='tight')

# Show the figure
plt.show()


In [ ]:
# Define Chi and G values more densely for a smooth surface
#Chi_values = np.linspace(0.0, 0.1, 2)  # Increased number of points for smoothness
#G_values = np.linspace(0.4, 0.4, 2)


# Create a meshgrid
#Chi_grid, G_grid = np.meshgrid(Chi_values, G_values)

print(f'Chi_grid: {Chi_grid}')
print(f'Chi_grid: {G_grid}')
print(f'U_grid: {U_grid}')

In [ ]:
# Define Chi and G values more densely for a smooth surface
Chi_values = np.linspace(0.0, 0.1, 2)  # Increased number of points for smoothness
G_values = np.linspace(0.4, 0.4, 2)


# Create a meshgrid
Chi_grid, G_grid = np.meshgrid(Chi_values, G_values)

# Initialize an empty grid for U values
U_grid = np.zeros_like(Chi_grid)

# Compute U values for each combination of Chi and G
for i in range(Chi_grid.shape[0]):
    for j in range(G_grid.shape[1]):

        par.Chi_ss = Chi_grid[i, j]
        par.G_ss = G_grid[i, j]
        model.find_ss(do_print=False)
        U_grid[i, j] = ss.U_hh

In [ ]:
# Plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Surface plot with color based on U values
surface = ax.plot_surface(Chi_grid, G_grid, U_grid, cmap='viridis')

# Adding a color bar
cbar = plt.colorbar(surface)
cbar.set_label('U values')

# Labeling axes
ax.set_xlabel('Chi')
ax.set_ylabel('G')
ax.set_zlabel('U')

# Show plot
plt.show()

# 3. Optimal welfare policies 2 (with transfers)

In [ ]:

# Define Chi and G values more densely for a smooth surface
#Chi_values = np.linspace(0.0, 0.1, 5)  # Increased number of points for smoothness
#G_values = np.linspace(0.3, 0.5, 5)

# Create a meshgrid
#Chi_grid, G_grid = np.meshgrid(Chi_values, G_values)

# Initialize an empty grid for U values
#U_grid = np.zeros_like(Chi_grid)

# Compute U values for each combination of Chi and G
#for i in range(Chi_grid.shape[0]):
#    for j in range(G_grid.shape[1]):
#        par.Chi_ = Chi_grid[i, j]
#        par.G_ = G_grid[i, j]
#        model.find_ss(do_print=False)
#        U_grid[i, j] = ss.U_hh

# Plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Surface plot with color based on U values
surface = ax.plot_surface(Chi_grid, G_grid, U_grid, cmap='viridis')

# Adding a color bar
cbar = plt.colorbar(surface)
cbar.set_label('U values')

# Labeling axes
ax.set_xlabel('Chi')
ax.set_ylabel('G')
ax.set_zlabel('U')

# Show plot
plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# ... [your code to compute Chi_list, G_list, U_list] ...

# Plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with color based on U values
scatter = ax.scatter(Chi_list, G_list, U_list, c=U_list, cmap='viridis')

# Adding a color bar
cbar = plt.colorbar(scatter)
cbar.set_label('U values')

# Labeling axes
ax.set_xlabel('Chi')
ax.set_ylabel('G')
ax.set_zlabel('U')

# Show plot
plt.show()

In [ ]:
# Optimizing over 3 dimensions
Chi_values = np.linspace(0.0, 0.5, 5)
G_values = np.linspace(0.3, 0.5, 5)
u_values = []

par.Chi_ss = i
par.G_ss = j 
model.find_ss(do_print=True)
ss.U_hh

## 2. <a id='toc2_'></a>[Test 1: Solving and simulating the household problem](#toc0_)

In [ ]:
model.compute_jacs(do_print=True)
model.test_jacs()

In [ ]:
model.test_path()

In [ ]:
model.find_transition_path(shocks=[],do_print=True)

## Question 2

In [ ]:
ss.U_hh

In [ ]:
#Works 
par.G_ss = 0.2
model.find_ss(do_print=False)
print(f'{ss.U_hh = :.2f}')
values = (0.0, 0.1, 0.15, 0.2, 0.25, 0.3)

#dosent work (says float cant be interpreted as an integer)
for i in values:
    print(type(i))
    par.G_ss = i
    model.find_ss(do_print=True)
    print(f'{ss.U_hh = :.2f}')
    print(f'{par.G_ss = :.2f}')
    print(f'{ss.L_G = :.2f}')

In [ ]:
values = (0.1, 0.15, 0.2, 0.25, 0.3)

for i in values:
    print(f'G = {i}')
    par.G_ss = i
    model.find_ss(do_print=False)
    print(f'Utility = {ss.U_hh:.2f}')

In [ ]:
# List of values
values = [0.001, 0.005, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5]

# Initialize empty lists to store inputs and utilities
inputs = []
utilities = []

# Iterate through the values
for i in values:
    print(f'G = {i}')
    par.G_ss = i
    model.find_ss(do_print=False)
    utility = ss.U_hh
    print(f'Utility = {utility:.2f}')
    print(f'{ss.L_G = :.2f}')
    print(f'{ss.L_Y = :.2f}')
    
    # Append the input value and utility to the respective lists
    inputs.append(i)
    utilities.append(utility)

# Plot the inputs and utilities
plt.figure(figsize=(8, 4))
plt.plot(inputs, utilities, marker='o', linestyle='-')
plt.xlabel('G')
plt.ylabel('Utility')
plt.title('Utility vs. G')
plt.grid(True)
plt.show()

And we can check whether the results make sort of sense:

In [ ]:
print(f'{model.ss.A_hh = :.2f}')
print(f'{model.ss.L_hh = :.2f}')
print(f'{model.ss.C_hh = :.2f}')

In [ ]:
model.test_hh_path()

## 3. <a id='toc3_'></a>[Test 2: Evaluating the objective for finding the steady state](#toc0_)

We can try out various inputs and look at the outputs:

In [ ]:
#Testting obj function 
# guessing on tau and K/L_y
x = [1.0, 0.0]

obj_ss(x, model)

## 4. <a id='toc4_'></a>[Find stationary equilibrium](#toc0_)

In [ ]:
model.find_ss(do_print=True)

## 5. <a id='toc5_'></a>[Grid search](#toc0_)

In [ ]:
par.tau_ss = 0.0
KL_min = ((1/par.beta+par.delta-1)/(par.alpha*par.Gamma_Y))**(1/(par.alpha-1))
KL_max = (par.delta/(par.alpha*par.Gamma_Y))**(1/(par.alpha-1))

In [ ]:
NKL = 10
KL_vec = np.hstack((np.linspace(KL_min+1e-2,KL_max-1e-2,NKL),np.linspace(KL_max+1e-2,10.0,NKL)))
clearing_A_vec = np.nan*np.ones(KL_vec.size)
r_vec = np.nan*np.ones(KL_vec.size)

model_ = model.copy()
for i,KL in enumerate(KL_vec):
    print(f'{KL = :6.2f}: ',end='')
    try:
        clearing_A_vec[i] = obj_ss(np.array([KL]),model_,do_print=False)
        r_vec[i] = model_.ss.r
        print(f'clearing_A = {clearing_A_vec[i]:16.8f}')
    except Exception as e:
        print(e)

In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)

# a. income
ax = fig.add_subplot(1,2,1)
ax.set_title('$B+K-A^{hh}$')
ax.plot(KL_vec[:NKL],clearing_A_vec[:NKL],'-o')
ax.plot(KL_vec[NKL:],clearing_A_vec[NKL:],'-o')
ax.axvline(ss.K/ss.L,color='black')
ax.set_yscale('symlog')

ax = fig.add_subplot(1,2,2)
ax.set_title('$r$')
ax.plot(KL_vec[:NKL],r_vec[:NKL],'-o')
ax.plot(KL_vec[NKL:],r_vec[NKL:],'-o')
ax.axvline(ss.K/ss.L,color='black');
ax.axhline(ss.r,color='black');

## 6. <a id='toc6_'></a>[Policy functions](#toc0_)

In [ ]:
i_fix = 0

fig = plt.figure(figsize=(18,4),dpi=100)
a_max = 50

# a. consumption
I = par.a_grid < a_max

ax = fig.add_subplot(1,3,1)
ax.set_title(f'consumption')

for i_z in [0,par.Nz//2,par.Nz-1]:
    ax.plot(par.a_grid[I],ss.c[i_fix,i_z,I],label=f'i_z = {i_z}')

ax.legend(frameon=True)
ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('consumption, $c_t$')

# b. saving
I = par.a_grid < a_max

ax = fig.add_subplot(1,3,2)
ax.set_title(f'saving')

for i_z in [0,par.Nz//2,par.Nz-1]:
    ax.plot(par.a_grid[I],ss.a[i_fix,i_z,I],label=f'i_z = {i_z}')

ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('savings, $a_{t}$')

# c. labor supply
I = par.a_grid < a_max

ax = fig.add_subplot(1,3,3)
ax.set_title(f'labor_supply')

for i_z in [0,par.Nz//2,par.Nz-1]:
    ax.plot(par.a_grid[I],ss.ell[i_fix,i_z,I],label=f'i_z = {i_z}')

ax.set_xlabel('savings, $a_{t-1}$')
ax.set_ylabel('labor supply, $n_{t}$')

fig.tight_layout()
plt.show()

## 7. <a id='toc7_'></a>[Simulation](#toc0_)

In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)

# a. income
ax = fig.add_subplot(1,2,1)
ax.set_title('productivity')

y = np.cumsum(np.sum(ss.D,axis=(0,2)))
ax.plot(par.z_grid,y/y[-1])

ax.set_xlabel('productivity, $z_{t}$')
ax.set_ylabel('CDF')

# b. assets
ax = fig.add_subplot(1,2,2)
ax.set_title('savings')
y = np.insert(np.cumsum(np.sum(ss.D,axis=(0,1))),0,0.0)
ax.plot(np.insert(par.a_grid,0,par.a_grid[0]),y/y[-1])
        
ax.set_xlabel('assets, $a_{t}$')
ax.set_ylabel('CDF')
ax.set_xscale('symlog')

## 8. <a id='toc8_'></a>[Test transition path](#toc0_)

In [ ]:
try:
    model.test_ss()
except Exception as e:
    print('you need to update GEModelTools to call this function (optional)')

In [ ]:
model.test_hh_path()

In [ ]:
model.draw_DAG()

In [ ]:
model.ss.tau

In [ ]:
model.test_path(in_place=True)

In [ ]:
model.path.A_hh

## 9. <a id='toc9_'></a>[Find transition path](#toc0_)

In [ ]:
model.compute_jacs(do_print=True)
model.find_transition_path(shocks=[],do_print=True)

In [ ]:
model.test_jacs()

In [ ]:
model.find_transition_path(shocks=[],do_print=True)

## 9. <a id='toc9_'></a>[Question a](#toc0_)

Calculating utility

In [ ]:
par.T = 500
v = np.sum([par.beta**t* np.sum(path.u[t,i_fix]*path.D[t,i_fix]/np.sum(path.D[t,i_fix]))
for t in range(par.T)])

print(f'{v = :.2f}')